In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import json
import sqlite3


In [3]:
#Database   Creation and  initilization 
conn = sqlite3.connect('airport.db')
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS flights(
               id INTEGER PRIMARY KEY,
               name TEXT NOT NULL,
               travelling TEXT NOT NULL,
               status TEXT NOT NULL)""")
conn.commit()
conn.close


<function Connection.close()>

In [8]:
#Funcation to insert the data into database 
def insert_into_database(passenger):
    conn = sqlite3.connect('airport.db')
    cursor = conn.cursor()
    cursor.execute(
    "INSERT INTO flights (id,name,travelling,status) VALUES (?,?,?,?)",
    passenger
    )
    conn.commit()
    conn.close()


In [9]:
data_passengers=[
        (101, "Ali", "Lahore to Karachi", "Not Booked"),
        (102, "Ahmed", "Lahore to Karachi", "Booked"),
        (103, "Hassan", "Lahore to Karachi", "Not Booked")
    ]


In [10]:
for d in data_passengers:
    insert_into_database(d)

In [27]:
#funcation to view the data,
def view_database():
    conn = sqlite3.connect('airport.db')
    cursor = conn.cursor()

    cursor.execute("" 
    "SELECT * FROM flights")
    rows = cursor.fetchall()
    conn.close() 
    for row in rows:
        print(row)
    return rows
      


In [5]:
view_database()

(101, 'Ali', 'Lahore to Karachi', 'Not Booked')
(102, 'Ahmed', 'Lahore to Karachi', 'Booked')
(103, 'Hassan', 'Lahore to Karachi', 'Not Booked')


In [20]:
#book a flight 

def booked_flight(status,id):
    conn = sqlite3.connect('airport.db')
    cursor = conn.cursor()

    cursor.execute("" 
    "UPDATE flights SET status = ? WHERE id = ? ",
    (status,id)
    )
    conn.commit()
    conn.close()


In [21]:
booked_flight('booked',101)

In [13]:
view_database()

(101, 'Ali', 'Lahore to Karachi', 'booked')
(102, 'Ahmed', 'Lahore to Karachi', 'Booked')
(103, 'Hassan', 'Lahore to Karachi', 'Not Booked')


In [22]:
#function to cancel the ticket
def cancel_ticket(status,id):
    conn = sqlite3.connect('airport.db')
    cursor = conn.cursor()
    
    cursor.execute(
        "UPDATE flights SET status =? WHERE id =?",
        ('NOT BOOKED',101)
    )
    conn.commit()
    conn.close()

    

In [23]:
cancel_ticket('NOT Booked',101)

In [24]:
view_database()

(101, 'Ali', 'Lahore to Karachi', 'NOT BOOKED')
(102, 'Ahmed', 'Lahore to Karachi', 'Booked')
(103, 'Hassan', 'Lahore to Karachi', 'Not Booked')


In [2]:
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv('GEMINI_API_KEY')


In [3]:
client = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key = OPENAI_API_KEY)

In [4]:
response = client.chat.completions.create(model = "gemini-2.5-flash",messages={'role':'user','content':'hellow'})

In [5]:
print(response.choices[0].message.content)

Hello! How can I help you today?


In [6]:
data_base = {'karachi to lahore':'plane name = Pk.2003 ,date=21 feb ,time = 11am','lahore to karachi':"plane name = pk.3000, date=22 feb,time=9am"}
def flight_detailes(Travelling):
    print(f'Tool calling for {Travelling}')
    plane_name = data_base.get(Travelling['Travelling'],"Unknow ticket")
    return f"Your travelling plane details is {plane_name}"



In [7]:
#funcation for booking 
def book_ticket(Travelling):
    print(f'Processing for booking ticket for {Travelling}')
    for key , value in data_base.items():
        if key == Travelling:
            #print("Your Ticket is booked")
            return f"your ticket booked and details of ticket is {data_base[Travelling]}"

In [52]:
#Defining the function to cancel the booked ticket 
def ticket_cancel(Travelling):
    print("Checking database for ticket to cancel")
    for key , val in data_base.items():
        if key == Travelling:
            return f"You are ticket from {Travelling} is cancelled !"

In [53]:
ticket_cancel('lahore to karachi')

Checking database for ticket to cancel


'You are ticket from lahore to karachi is cancelled !'

In [8]:
print(book_ticket('karachi to lahore'))

Processing for booking ticket for karachi to lahore
your ticket booked and details of ticket is plane name = Pk.2003 ,date=21 feb ,time = 11am


In [28]:
viewAll_data = {
    'name':'view_database()',
    'description':'Call this function when user went see all of the database',
    'parameters':{
        'type':'object',
        "properties":{},
        'required':[]
    }
    
}

In [31]:
booked_tool = {
    "name":'booked_flight',
    "description":"User this tool when user went to booked the flight",
    'parameters':{
        'type':'object',
        "properties":{
            'status':{
                'type':'string',
                'description':'This parameter represent the  status in database they booked or not'
            },
            'id':{
             'type':'integer',
             'description':'This parameters is Primary key of user , using this we access the user data and update it'
            }
        },
        'required':['status','id']
    }
}

In [32]:
ticket_cancel_tool = {
    "name":'cancel_ticket',
    'description':'Used this function when user went to cancel their booked ticket',
    'parameters':{
        'type':'object',
        'properties':{
            'status':{
                'type':'string',
                'description':'The parameter required to update the user status form booked to NOT-Booked'
            },
            'id':{
                'type':'integer',
                'description': 'This is paramter is primary key , using this we acces the user data from database'

            },    
        },
        'required': ['status','id'],
    }
    
}

In [35]:
#tool = tool1
tool = [viewAll_data,booked_tool,ticket_cancel_tool]

In [36]:
tools = {'type':'function','function':tool}

In [37]:
system_message = """ You are an intelliegent Airport helper Agent  your tasked is to reply the user base on their input :
input are of three type: 

1nd type = user : my id is 111, I went to travelled from lahore to karachi book the flight for me
assistent : Congratulation i will  book the ticket for you , from you source(given by user)  to destination(given in used input)

2st type = user :my id is 222 I went to cancel my ticket, 
assitent : your  id , and i cancel the flight form database

3nd type = user : I went to see all of the database  
assistent : here is the all database 

 """

In [56]:
def handle_tool(content):
    content1 = content.tool_calls[0]
    if content1.function.name == 'flight_detailes':
        arg = json.loads(content1.function.arguments)
        result = flight_detailes(arg)
        response ={
            'role':'tool',
            'content':result,
            "tool_call_id": content1.id
        }
    if content1.function.name == 'book_ticket':
        arg = json.loads(content1.function.arguments)
        result = book_ticket(arg)
        response = {'role':'tool','content':result,'tool_call_id':content1.id}

    if content1.function.name == 'ticket_cancel':
        args = json.loads(content1.function.arguments)
        result = ticket_cancel(args) 
        response = {'role':'tool','content':result,'tool_call_id':content1.id} 
          

    return response    
    

In [57]:
def chat_with_ai(message,history):
    history = [{'role':h['role'],'content':h['content']}for h in history]
    message = [{'role':'system','content':system_message}] + history + [{"role":'user',"content":message}]
    response = client.chat.completions.create(model='gemini-2.5-flash',messages=message,tools=tools)

    if response.choices[0].finish_reason=='tool_calls': 
        content = response.choices[0].message
        tool_call1 = content.tool_calls[0]
        if tool_call1.function.name == 'flight_detailes':
            print('Tool calling for Flight detailes')
            result = handle_tool(content)
            print('Coming back to llm with fligh details')
            message.append(content)
            message.append(result)
            print('Second call to LLm')
            response = client.chat.completions.create(model='gemini-2.5-flash',messages=message,tools=tools)
        if tool_call1.function.name == "book_ticket":
            print('Tool calling for the booking ticket')
            result = handle_tool(content)
            print('Comming back after booking ticket')
            message.append(content)
            message.append(result)
            print('Making the second time call')
            response = client.chat.completions.create(model='gemini-2.5-flash',messages=message,tools=tools)
        if tool_call1.function.name == 'ticket_cancel':
            print('Calling the tool for cancelling the ticket')
            result = handle_tool(content)        
            print('Coming back after cancelling ticket')
            message.append(content)
            message.append(result)
            response = client.chat.completions.create(model='gemini-2.5-flash',messages=message,tools=tools)

    return response.choices[0].message.content    

        
    

In [35]:
import gradio as gr

In [58]:
gr.ChatInterface(fn=chat_with_ai,type='messages').launch(share=True)

* Running on local URL:  http://127.0.0.1:7872
* Running on public URL: https://78e38046387d7e8b66.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
